In [1]:
import pandas as pd
import string

In [2]:
# Method to combine main city data file with AEA file
def aeaMerge(file1, file2):  # File1 should probably be the CD_CSV file mostly
    df_file1 = pd.DataFrame(pd.read_csv(file1, sep=',', encoding='utf-8'))
    df_file2 = pd.DataFrame(pd.read_csv(file2, sep=',', encoding='utf-8'))

    # Check to make sure correct second file was passed in
    if file2 == 'files/Fully_Cleaned_AEA_Data.csv':
        # By making this new DataFrame I cut out the excess index column from the second file.
        stats_from_file_two = pd.DataFrame(data=[df_file2['County'], df_file2['Higher Degree'], df_file2['H.S Diploma'], df_file2['No H.S Diploma']]).transpose()
        df_output = pd.merge(df_file1, stats_from_file_two, on='County')
        df_output.to_csv('files/CD_and_Statistical_Atlas_Data_Combined.csv', sep=',', encoding='utf-8')

In [3]:
# Method to combine all of the zip code level data (from cities and counties) into the main City Data csv file
def combinezips(file1, file2):
    # Create dfs for the two zip code related files
    df_file1 = pd.DataFrame(pd.read_csv(file1, sep=',', encoding='utf-8'))
    df_file2 = pd.DataFrame(pd.read_csv(file2, sep=',', encoding='utf-8'))

    df_file2['ZipCode'] = df_file2['ZipCode'].astype('int32')

    # Merge two dfs and then create a new sub-dataframe that contains just the columns we care about
    combined = pd.merge(df_file1, df_file2, on='ZipCode')
    combined_main_columns = pd.DataFrame(data=[combined['City'], combined['ZipCode'], combined['Lat'], combined['Long']]).transpose()
    combined_main_columns['ZipCode'] = combined_main_columns['ZipCode'].astype('int32')

    # Load in the main file so it can have zip code data merged into it
    main_file = pd.DataFrame(pd.read_csv('files/CD_and_Statistical_Atlas_Data_Combined.csv', sep=',', encoding='utf-8'))
    main_file = main_file.rename(index=str, columns={"City Name": "City"})

    updated_main = pd.merge(main_file, combined_main_columns, on='City')

    # Code to deal with smaller cities that didn't get a zip code on web site we scrapped
    # Resolved by giving them county level lat/long and zip
    blank_subset = pd.merge(main_file, combined_main_columns, on='City', how='left')
    blank_subset['ZipCode'] = blank_subset['ZipCode'].fillna(0)
    blank_subset = blank_subset[blank_subset['ZipCode'] == 0]

    # Load in the county zip files to fill in the gaps
    county_zip1 = pd.DataFrame(pd.read_csv('files/County_Zip.csv', sep=',', encoding='utf-8'))
    county_zip2 = pd.DataFrame(pd.read_csv('files/County_Zip_Lat_Long.csv', sep=',', encoding='utf-8'))
    county_zip2['ZipCode'] = county_zip2['ZipCode'].astype('int32')
    combined_county_zip = pd.merge(county_zip1, county_zip2, on='ZipCode')
    combined_county_zip['County'] = combined_county_zip['County'] + '-County'
    blank_subset_filled = pd.merge(blank_subset, combined_county_zip, on='County')

    # Clean out 2 useless columns
    updated_main = updated_main.drop(updated_main.columns[0], axis=1)
    updated_main = updated_main.drop(updated_main.columns[0], axis=1)
    blank_subset_filled = blank_subset_filled.drop(blank_subset_filled.columns[0], axis=1)
    blank_subset_filled = blank_subset_filled.drop(blank_subset_filled.columns[0], axis=1)

    # Clean out additional junk columns that were produced by merge calls
    del blank_subset_filled['Unnamed: 0_y']
    del blank_subset_filled['Unnamed: 0_x']
    del blank_subset_filled['Long_x']
    del blank_subset_filled['Lat_x']
    del blank_subset_filled['ZipCode_x']

    # Reformatted column types
    blank_subset_filled = blank_subset_filled.rename(index=str, columns={'Lat_y': 'Lat', 'Long_y': 'Long', 'ZipCode_y': 'ZipCode'})
    blank_subset_filled['ZipCode'] = blank_subset_filled['ZipCode'].astype('int32')
    blank_subset_filled['Lat'] = blank_subset_filled['Lat'].astype('object')
    blank_subset_filled['Long'] = blank_subset_filled['Long'].astype('object')

    df = [updated_main, blank_subset_filled]

    # Concatenate together cities that got zip code from city-zip file and county-zip file
    final_dataframe = pd.concat(df)
    final_dataframe.to_csv('files/CD_SA_with_Zip_all.csv', sep=',', encoding='utf-8')

In [4]:
# Method to combine all of the zip code level data (from cities and counties) into the main City Data csv file
def combinezips(file1, file2):
    # Create dfs for the two zip code related files
    df_file1 = pd.DataFrame(pd.read_csv(file1, sep=',', encoding='utf-8'))
    df_file2 = pd.DataFrame(pd.read_csv(file2, sep=',', encoding='utf-8'))

    df_file2['ZipCode'] = df_file2['ZipCode'].astype('int32')

    # Merge two dfs and then create a new sub-dataframe that contains just the columns we care about
    combined = pd.merge(df_file1, df_file2, on='ZipCode')
    combined_main_columns = pd.DataFrame(data=[combined['City'], combined['ZipCode'], combined['Lat'], combined['Long']]).transpose()
    combined_main_columns['ZipCode'] = combined_main_columns['ZipCode'].astype('int32')

    # Load in the main file so it can have zip code data merged into it
    main_file = pd.DataFrame(pd.read_csv('files/CD_and_Statistical_Atlas_Data_Combined.csv', sep=',', encoding='utf-8'))
    main_file = main_file.rename(index=str, columns={"City Name": "City"})

    updated_main = pd.merge(main_file, combined_main_columns, on='City')

    # Code to deal with smaller cities that didn't get a zip code on web site we scrapped
    # Resolved by giving them county level lat/long and zip
    blank_subset = pd.merge(main_file, combined_main_columns, on='City', how='left')
    blank_subset['ZipCode'] = blank_subset['ZipCode'].fillna(0)
    blank_subset = blank_subset[blank_subset['ZipCode'] == 0]

    # Load in the county zip files to fill in the gaps
    county_zip1 = pd.DataFrame(pd.read_csv('files/County_Zip.csv', sep=',', encoding='utf-8'))
    county_zip2 = pd.DataFrame(pd.read_csv('files/County_Zip_Lat_Long.csv', sep=',', encoding='utf-8'))
    county_zip2['ZipCode'] = county_zip2['ZipCode'].astype('int32')
    combined_county_zip = pd.merge(county_zip1, county_zip2, on='ZipCode')
    combined_county_zip['County'] = combined_county_zip['County'] + '-County'
    blank_subset_filled = pd.merge(blank_subset, combined_county_zip, on='County')

    # Clean out 2 useless columns
    updated_main = updated_main.drop(updated_main.columns[0], axis=1)
    updated_main = updated_main.drop(updated_main.columns[0], axis=1)
    blank_subset_filled = blank_subset_filled.drop(blank_subset_filled.columns[0], axis=1)
    blank_subset_filled = blank_subset_filled.drop(blank_subset_filled.columns[0], axis=1)

    # Clean out additional junk columns that were produced by merge calls
    del blank_subset_filled['Unnamed: 0_y']
    del blank_subset_filled['Unnamed: 0_x']
    del blank_subset_filled['Long_x']
    del blank_subset_filled['Lat_x']
    del blank_subset_filled['ZipCode_x']

    # Reformatted column types
    blank_subset_filled = blank_subset_filled.rename(index=str, columns={'Lat_y': 'Lat', 'Long_y': 'Long', 'ZipCode_y': 'ZipCode'})
    blank_subset_filled['ZipCode'] = blank_subset_filled['ZipCode'].astype('int32')
    blank_subset_filled['Lat'] = blank_subset_filled['Lat'].astype('object')
    blank_subset_filled['Long'] = blank_subset_filled['Long'].astype('object')

    df = [updated_main, blank_subset_filled]

    # Concatenate together cities that got zip code from city-zip file and county-zip file
    final_dataframe = pd.concat(df)
    final_dataframe.to_csv('files/CD_SA_with_Zip_all.csv', sep=',', encoding='utf-8')

In [5]:
# Method to combine all the remaining educational data into our master file
def male_female_age_edu_combiner():
    medu = pd.DataFrame(pd.read_csv('files/ADEASexRatioMale.csv', sep=',', encoding='utf-8'))
    fedu = pd.DataFrame(pd.read_csv('files/ADEASexRatioFemale.csv', sep=',', encoding='utf-8'))
    aedu = pd.DataFrame(pd.read_csv('files/AllBachDegreeByAge.csv', sep=',', encoding='utf-8'))

    # Grab the columns for one last cleaning
    mcolumns = medu.columns
    fcolumns = fedu.columns
    acolumns = aedu.columns

    # Loop through (skipping the first column) and remove % signs from the data for each data frame
    counter = 0
    for col in mcolumns:
        if counter == 1:
            medu[col] = medu[col].str.replace('%', '')
        else:
            counter += 1
    counter = 0
    for col in fcolumns:
        if counter == 1:
            fedu[col] = fedu[col].str.replace('%', '')
        else:
            counter += 1
    counter = 0
    for col in acolumns:
        if counter == 1:
            aedu[col] = aedu[col].str.replace('%', '')
        else:
            counter += 1

    # Load in enthnicity.csv which is our most complete file at this point
    masterfile = pd.DataFrame(pd.read_csv('files/ethnicity.csv', sep=',', encoding='utf-8'))

    # Merge int additional data
    masterfile = pd.merge(masterfile, medu, on='County')
    masterfile = pd.merge(masterfile, fedu, on='County')
    masterfile = pd.merge(masterfile, aedu, on='County')

    # Remove unneeded columns
    masterfile = masterfile.drop(masterfile.columns[0], axis=1)
    masterfile = masterfile.drop('Unnamed: 0_y', axis=1)
    masterfile.to_csv('files/Master_Data_File.csv', sep=',', encoding='utf-8')

In [6]:
# Method to combine the two school related data sources together / not in use actually
def combine_school_data():
    file1 = pd.DataFrame(pd.read_csv('Summarized_School_Scores.csv', sep=',', encoding='utf-8'))
    file2 = pd.DataFrame(pd.read_csv('School_Safety_Data.csv', sep=',', encoding='utf-8'))

    df = pd.merge(file1, file2, on='School Number')
    # Drop unneeded columns
    df = df.drop(df.columns[0:2], axis=1)
    df = df.drop(df.columns[6:8], axis=1)
    df.to_csv('School_Data_Combined.csv', sep=',', encoding='utf-8')

In [7]:
def remove_punct(text):
    txt = "".join([char for char in text if char not in string.punctuation])
    return txt 

In [8]:
def extract(text):
    if 'White alone' in text:
        txt = text.split('White alone ')[1]
        txtw = txt.split(' ')[0]
    else:
        txtw = '0'
    if 'Hispanic' in text:
        txt = text.split('Hispanic ')[1]
        txth = txt.split(' ')[0]
    else:
        txth = '0'
    if 'Black alone' in text:
        txt = text.split('Black alone ')[1]
        txtb = txt.split(' ')[0] 
    else:
        txtb = '0'
    if 'Asian alone' in text: 
        txt = text.split('Asian alone ')[1]
        txta = txt.split(' ')[0]
    else:
        txta = '0'
    if 'American Indian alone' in text:
        txt = text.split('American Indian alone ')[1]
        txtai = txt.split(' ')[0]
    else:
        txtai = '0'
    if 'Two or more races' in text:
        txt = text.split('Two or more races ')[1]
        txttm = txt.split(' ')[0]
    else:
        txttm = '0'
    if 'Other race alone' in text:
        txt = text.split('Other race alone ')[1]
        txtoth = txt.split(' ')[0]
    else:
        txtoth = '0'
    if 'Native Hawaiian and Other Pacific Islander alonebr' in text:
        txt = text.split('Native Hawaiian and Other Pacific Islander alonebr ')[1]
        txtnh = txt.split(' ')[0]
    elif 'Native Hawaiian and Other Pacific Islander alone' in text:
        txt = text.split('Native Hawaiian and Other Pacific Islander alone ')[1]
        txtnh = txt.split(' ')[0]
    elif 'Native Hawaiian and Other  Pacific Islander alone' in text:
        txt = text.split('Native Hawaiian and Other  Pacific Islander alone ')[1]
        txtnh = txt.split(' ')[0]   
    else:
        txtnh ='0'
    return txtw, txth, txtb, txta, txtai, txttm, txtoth, txtnh 

In [9]:
#Hey Kaurik you need to methodize your code here
def extract_ethnicity():
    # Import Pandas
    import pandas as pd 
    # Read File to extract columns for ethinic data 
    data = pd.read_csv('files/CD_SA_with_Zip_all.csv')
    # Import numpy 

    import numpy as np

    # Set all Ethnic columns to null
    data['White alone'] = np.nan
    data['Hispanic'] = np.nan
    data['Two or more races'] = np.nan
    data['Native Hawaiian'] = np.nan
    data['Black alone'] = np.nan
    data['Asian alone'] = np.nan
    data['American Indian alone'] = np.nan
    data['Other race alone'] = np.nan
    
    # dropping impure rows
    data.drop(data.index[np.where(data['Racial Breakdown'] == '*')], inplace=True)
    # Deleting the initial additional index 
    del data['Unnamed: 0']
    # Create a copy 
    data1 = data.copy()
    # Reset index
    data1.reset_index(inplace=True)
    
    i=0
    for index, row in data.iterrows():
        if i==1641:
            break
        line = row['Racial Breakdown']
        clean_text = remove_punct(line)
        w, h, b, a, ai, tm, oth, nh = extract(clean_text)
        data1.at[i, 'White alone'] = w
        data1.at[i, 'Hispanic'] = h
        data1.at[i, 'Two or more races'] = tm
        data1.at[i, 'Native Hawaiian'] = nh
        data1.at[i, 'Black alone'] = b
        data1.at[i, 'Asian alone'] = a
        data1.at[i, 'American Indian alone'] = ai
        data1.at[i, 'Other race alone'] = oth
        i = i + 1
    
    # Remove additional index column
    del data1['index']
    
    # Create a CSV for further processing
    data1.to_csv('files/ethnicity.csv', sep=',', encoding='utf-8')

In [10]:
# Make sure to update what files you are passing in for combination. At this point,
# CD_and_Statistical_Atlas_Data_Combined is the master file currently
if __name__ == '__main__':
    aeaMerge('files/Fully_Cleaned_CD_Data.csv', 'files/Fully_Cleaned_AEA_Data.csv')
    combinezips('files/City_Zip.csv', 'files/City_Zip_Lat_Long.csv')
    
    extract_ethnicity()
    
    male_female_age_edu_combiner()